In [2]:
import pandas as pd 
import numpy as np 
import torch
import networkx as nx
import matplotlib.pyplot as plt
from text_preprocess import preprocess_text



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ash_3\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df_15_train=pd.read_csv(r'dataset\twitter15\twitter15.train',sep='\t',names=['tweet_id','context','label'])
df_15_test=pd.read_csv(r'dataset\twitter15\twitter15.test',sep='\t',names=['tweet_id','context','label'])
df_15_dev=pd.read_csv(r'dataset\twitter15\twitter15.dev',sep='\t',names=['tweet_id','context','label'])
df_15_total=pd.concat([df_15_train,df_15_dev,df_15_test],ignore_index=True)
df_15_total.shape

(1490, 3)

In [4]:
df_15_total.head()

,tweet_id,context,label
0,724703995147751424,"american family association gets 500,000 to si...",unverified
1,358591089462099968,this week's top story: george zimmerman wins f...,false
2,775672628493357057,clinton hides failing health? full disclosure ...,unverified
3,364589696573124609,fukushima: highly radioactive water seeping in...,false
4,549927969032916993,a transgender 17-year old left a suicide note ...,unverified


In [5]:
df_15_total['label'].value_counts()

label
non-rumor     374
unverified    373
true          372
false         371
Name: count, dtype: int64

In [6]:
df_15_total['class']=df_15_total['label'].replace({'false':0,'true':1,'unverified':2,'non-rumor':3})

df_15_total.head()

,tweet_id,context,label,class
0,724703995147751424,"american family association gets 500,000 to si...",unverified,2
1,358591089462099968,this week's top story: george zimmerman wins f...,false,0
2,775672628493357057,clinton hides failing health? full disclosure ...,unverified,2
3,364589696573124609,fukushima: highly radioactive water seeping in...,false,0
4,549927969032916993,a transgender 17-year old left a suicide note ...,unverified,2


In [10]:
df_15_total['tokens']=df_15_total['context'].apply(preprocess_text)
df_15_total.head()

,tweet_id,context,label,class,tokens
0,724703995147751424,"american family association gets 500,000 to si...",unverified,2,american famili associ get 500000 sign petit b...
1,358591089462099968,this week's top story: george zimmerman wins f...,false,0,week top stori georg zimmerman win florida sta...
2,775672628493357057,clinton hides failing health? full disclosure ...,unverified,2,clinton hide fail health full disclosur covert...
3,364589696573124609,fukushima: highly radioactive water seeping in...,false,0,fukushima highli radioact water seep ocean url
4,549927969032916993,a transgender 17-year old left a suicide note ...,unverified,2,transgend 17year old left suicid note tumblr p...


Applying TFIDF on the processed text

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer()

tfidf_matrix = vectorizer.fit_transform(df_15_total['tokens'])

# Convert the TF-IDF matrix to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Concatenate the original DataFrame with the TF-IDF DataFrame

result_df = pd.concat([df_15_total, tfidf_df], axis=1)

tfidf_df.head()

,10,100,10000,1030,11,1201,1214,129,12th,12yearold,...,ดว,นทหารย,นเคร,องบ,าใจผ,เครนร,เครนอาจเข,เป,เผยกบฏย,เพ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
tfidf_df.shape

(1490, 4063)

In [14]:
result_df.head()

,tweet_id,context,label,class,tokens,10,100,10000,1030,11,...,ดว,นทหารย,นเคร,องบ,าใจผ,เครนร,เครนอาจเข,เป,เผยกบฏย,เพ
0,724703995147751424,"american family association gets 500,000 to si...",unverified,2,american famili associ get 500000 sign petit b...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,358591089462099968,this week's top story: george zimmerman wins f...,false,0,week top stori georg zimmerman win florida sta...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,775672628493357057,clinton hides failing health? full disclosure ...,unverified,2,clinton hide fail health full disclosur covert...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,364589696573124609,fukushima: highly radioactive water seeping in...,false,0,fukushima highli radioact water seep ocean url,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,549927969032916993,a transgender 17-year old left a suicide note ...,unverified,2,transgend 17year old left suicid note tumblr p...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
